In [1]:
SPARK_VERSION = '3.1.2'
SCALA_VERSION = '2.12'

import findspark
findspark.add_packages(['org.apache.spark:spark-sql-kafka-0-10_' + SCALA_VERSION + ':' + SPARK_VERSION ])

findspark.init()
 
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
sc = SparkContext(appName="PythonSparkStreamingKafka")
ssc = StreamingContext(sc, 20)
topicPartion = ('saulo-teste',0)
fromOffset = {topicPartion: 0}

##lines = KafkaUtils.createDirectStream(ssc, ['saulo-teste'],{"bootstrap.servers": 'localhost:9092'}, fromOffsets=fromOffset)

##lines.pprint()

##ssc.start()
##ssc.awaitTermination()



:: loading settings :: url = jar:file:/home/saulos_pires/.local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/saulos_pires/.ivy2/cache
The jars for the packages stored in: /home/saulos_pires/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-47338e39-b4da-4f4d-9826-9b64fcf1f614;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 329ms :: artifacts dl 16ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 

In [3]:

from delta import *

builder = pyspark.sql.SparkSession.builder.appName("AppSaulo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.debug.maxToStringFields", 100000)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [4]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "saulo-teste") \
  .load()
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

DataFrame[key: string, value: string]

In [5]:
print(df)

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]


In [7]:
df.select("topic")

DataFrame[topic: string]